In [1]:
import numpy as np

In [22]:
beta = 0.8
tol = 10**-6

In [2]:
def getGraphData(file):
    graphEdges = dict()
    with open(file) as fp:
        line = fp.readline()
        while line:
            lineL = [ int(node) for node in line.strip().split('\t')]
            if graphEdges.get(lineL[0]):
                graphEdges[lineL[0]].append(lineL[1])
            else:
                graphEdges[lineL[0]] = [lineL[1]]
            line = fp.readline()
    return graphEdges

graphEdges = getGraphData('web-Stanford.txt')


In [21]:
numberOfNodes = sorted(graphEdges.keys(), reverse=True)[:1][0]
print("Number of nodes using is", numberOfNodes)

Number of nodes using is 281903


In [20]:
def createMatrixFromGraph(graph, size):
    matrix = np.zeros((size, size))
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1

        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            count = valDict.get(nodeVal)
            matrix[nodeVal-1][node-1] = count * float(eachNodeVal)
    return matrix

matrix = createMatrixFromGraph(graphEdges, numberOfNodes)